In [31]:
# Cleanup some fields that are noew duplicated or are useless

import pandas as pd
import json

# circuits - none - but deal with lat and lng
# constructor_standings - none
# constructors - none
# driver_standings - none
# drivers - none
# lap_times - delete time field - we already have milliseconda
# pit_stops - delete time field - already joined in datetime field
# qualifying - delete q1, q2, q3 - we already have milliseconds
# races - delete date, time, and both fields for fp1, fp2, fp3, quali and sprint - all joined in datetime fields
# results - delete time and fastestLapTime - we already have milliseconds
# sprint_results - delete time and fastestLapTime - we already have milliseconds 
# seasons - none but add the driver_changes and reg_changes columns
# status - none

cleanup_path = './db_cleanup/'
export_path = './f1db_json/'

#lap_times 

lap_times = pd.read_json(cleanup_path + 'lap_times.json')
lap_times = lap_times.drop(columns=['time'])
lap_times.to_json(export_path + 'lap_times.json', orient='records')

# pit_stops

pit_stops = pd.read_json(cleanup_path + 'pit_stops_datetime.json')
pit_stops = pit_stops.drop(columns=['time'])
pit_stops.to_json(export_path + 'pit_stops.json', orient='records')

# qualifying

qualifying = pd.read_json(cleanup_path + 'qualifying_ms.json')
qualifying = qualifying.drop(columns=['q1', 'q2', 'q3'])
qualifying.to_json(export_path + 'qualifying.json', orient='records')

# races

races = pd.read_json(cleanup_path + 'races_datetime.json')
races = races.drop(columns=['date', 'time', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'])
races.to_json(export_path + 'races.json', orient='records')

# results

results = pd.read_json(cleanup_path + 'results_ms.json')
results = results.drop(columns=['time', 'fastestLapTime'])
results.to_json(export_path + 'results.json', orient='records')

# sprint_results

sprint_results = pd.read_json(cleanup_path + 'sprint_results_ms.json')
sprint_results = sprint_results.drop(columns=['time', 'fastestLapTime'])
sprint_results.to_json(export_path + 'sprint_results.json', orient='records')

# seasons

cleanup_path = './db_cleanup/'
wiki_data = "./wiki_scraping/"

seasons = pd.read_json(cleanup_path + 'seasons.json')
driver_changes_json = json.loads(open(wiki_data + 'f1_driver_changes.json').read())
reg_changes_json = json.loads(open(wiki_data + 'f1_regulation_changes.json').read())

# add the new columns to seasons, each column must contain the text for the equivalent year

driver_changes = []
reg_changes = []

for year in driver_changes_json:
    driver_changes.append(driver_changes_json[year]['Driver changes'])
    reg_changes.append(reg_changes_json[year]['Regulation changes'])

#instead of lists, use a dict with key as year and value as the text

driver_changes_dict = {}
reg_changes_dict = {}

for i in range(len(driver_changes)):
    driver_changes_dict[str(i + 1990)] = driver_changes[i]
    reg_changes_dict[str(i + 1990)] = reg_changes[i]
    
for index, row in seasons.iterrows():
    year = str(row['year'])
    driver_changes = driver_changes_dict.get(year, None)
    reg_changes = reg_changes_dict.get(year, None)

    if driver_changes == 'None relevant':
        driver_changes = None

    if reg_changes == 'None relevant':
        reg_changes = None

    if driver_changes is not None:
        driver_changes = '\n'.join(driver_changes)
    
    if reg_changes is not None:
        reg_changes = '\n'.join(reg_changes)
        
    seasons.at[index, 'driver_changes'] = driver_changes
    seasons.at[index, 'reg_changes'] = reg_changes 

seasons.to_json(cleanup_path + 'seasons_data.json', orient='records')
seasons.to_json(export_path + 'seasons_data.json', orient='records')


/var/folders/zt/0y3b2ss90sqbg4jfp0dmxf9w0000gn/T/ipykernel_44036/1802559417.py:103: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'The only offseason driver change was following the retirement of Red Bull's David Coulthard after 14 years in Formula One. He was replaced by Sebastian Vettel, who had raced for Toro Rosso in 2008. Vettel's seat at Toro Rosso was taken by the Swiss driver Sébastien Buemi, who was Red Bull's test driver in 2008.
Following the German Grand Prix, Toro Rosso's Sébastien Bourdais was dropped by the team, with Toro Rosso principal Franz Tost claiming that the partnership had not met his expectations. Bourdais was replaced by Jaime Alguersuari ahead of the Hungarian Grand Prix. Alguersuari had been racing in Formula Renault 3.5 Series in 2009, and had only signed a deal to replace Brendon Hartley as Toro Rosso's test driver two weeks prior. Bourdais was advised by counsel to file suit for breac